Predicting life time value for drivers based on historical behaviour data.
By - Pramod Chavan



In [ ]:
import pandas as pd
from datetime import datetime as dt, timedelta
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from pytz import timezone
import pytz

# 1) Overview of datasets

## a) driver_ids.csv dataset

In [ ]:
df_driver_ids = pd.read_csv("C:\Personal\Jobs\Lyft Data Science Assignment\data\driver_ids.csv")
print('Driver_ids dataframe dimension:', df_driver_ids.shape)

Driver_ids dataframe dimension: (937, 2)


In [ ]:
df_info = pd.DataFrame(df_driver_ids.dtypes).T.rename(index={0:'column_type'})
df_info = df_info.append(pd.DataFrame(df_driver_ids.nunique()).T.rename(index={0:'count_unique_value'}))
df_info = df_info.append(pd.DataFrame(df_driver_ids.min()).T.rename(index={0:'min_value'}))
df_info = df_info.append(pd.DataFrame(df_driver_ids.max()).T.rename(index={0:'max_value'}))
df_info = df_info.append(pd.DataFrame(df_driver_ids.isnull().sum()).T.rename(index={0:'null_values'}))
df_info

,driver_id,driver_onboard_date
column_type,object,object
count_unique_value,937,49
min_value,002be0ffdc997bd5c50703158b7c2491,2016-03-28 00:00:00
max_value,ffff51a71f2f185ec5e97d59dbcd7a78,2016-05-15 00:00:00
null_values,0,0


Observations: <br>
1. Contains drivers onboarding information <br>
2. Unique value of 937 for driver_id indicates that we don't have duplicate onboarding information for drivers <br>
3. We have driver's information on-boarded between 28th March 2016 and 15th May 2016 <br>
4. There are no missing values for driver_ids and driver_onboard_date <br>

## b) ride_ids.csv dataset

In [ ]:
df_ride_ids = pd.read_csv("C:\Personal\Jobs\Lyft Data Science Assignment\data\\ride_ids.csv")
print('Rider_ids dataframe dimension:', df_ride_ids.shape)

Rider_ids dataframe dimension: (193502, 5)


In [ ]:
df_info = pd.DataFrame(df_ride_ids.dtypes).T.rename(index={0:'column_type'})
df_info = df_info.append(pd.DataFrame(df_ride_ids.nunique()).T.rename(index={0:'count_unique_value'}))
df_info = df_info.append(pd.DataFrame(df_ride_ids.min()).T.rename(index={0:'min_value'}))
df_info = df_info.append(pd.DataFrame(df_ride_ids.max()).T.rename(index={0:'max_value'}))
df_info = df_info.append(pd.DataFrame(df_ride_ids.isnull().sum()).T.rename(index={0:'null_values'}))
df_info

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
column_type,object,object,int64,int64,int64
count_unique_value,937,193502,28259,4092,12
min_value,002be0ffdc997bd5c50703158b7c2491,00003037a262d9ee40e61b5c0718f7f0,-2,2,0
max_value,ffff51a71f2f185ec5e97d59dbcd7a78,ffffccd77f47a3de26dfed9a851464b4,724679,28204,500
null_values,0,0,0,0,0


Observations: <br>
1. Contains information for 193502 unique rides and 937 unique  drivers (same as driver ids)
2. Negative value for ride_distance seems to be incorrect data. However we have only record so it has been deleted.
3. No missing values present

## c) ride_timestamps.csv dataset

In [ ]:
df_ride_timestamps = pd.read_csv("C:\Personal\Jobs\Lyft Data Science Assignment\data\\ride_timestamps.csv")
print(df_ride_timestamps.shape)

(970405, 3)


In [ ]:
df_info = pd.DataFrame(df_ride_timestamps.dtypes).T.rename(index={0:'column_type'})
df_info = df_info.append(pd.DataFrame(df_ride_timestamps.nunique()).T.rename(index={0:'unique_value'}))
df_info = df_info.append(pd.DataFrame(df_ride_timestamps.isnull().sum()).T.rename(index={0:'null_values'}))
df_info

,ride_id,event,timestamp
column_type,object,object,object
unique_value,194081,5,865826
null_values,0,0,1


Observations:<br>
1. Unique value of 194081 for ride_id in ride_timestamps indicates that we have few records missing in ride_ids dataset.

# 2) Data Cleaning

> 1. Ride Ids dataset has negative value in ride distance column. Ride distance cannot be negative. We have only one record with negative distance and that record is removed

In [ ]:
df_ride_ids[(df_ride_ids[['ride_distance']] < 0).all(1)]

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
68704,569271b82c506166a23ffcfbf6f854f2,ff1ca66337a18055f1c70c7645399972,-2,181,50


In [ ]:
df_ride_ids = df_ride_ids[(df_ride_ids[['ride_distance']] >= 0).all(1)]
print('New Rider_ids dataframe dimension:', df_ride_ids.shape)

New Rider_ids dataframe dimension: (193501, 5)


>2. Ride timestamp contains only one record with null timestamp. As its just one record we will delete the ride associated with it

In [ ]:
df_ride_timestamps[df_ride_timestamps.isnull().any(axis=1)]

,ride_id,event,timestamp
434222,72f0fa0bd86800e9da5c4dced32c8735,arrived_at,NaN


In [ ]:
df_ride_timestamps = df_ride_timestamps[df_ride_timestamps.ride_id != '72f0fa0bd86800e9da5c4dced32c8735']
df_ride_timestamps.shape

(970400, 3)

>3. Timestamp in ride timestamp dataset is in UTC. As rides in dataset occured in SF we will convert the timestamps to PST timezoe

In [ ]:
df_ride_timestamps['timestamp'] = pd.to_datetime(df_ride_timestamps['timestamp'])
df_ride_timestamps['timestamp'] = df_ride_timestamps['timestamp'].dt.tz_localize('utc').dt.tz_convert('US/Pacific')
df_ride_timestamps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 970400 entries, 0 to 970404
Data columns (total 3 columns):
ride_id      970400 non-null object
event        970400 non-null object
timestamp    970400 non-null datetime64[ns, US/Pacific]
dtypes: datetime64[ns, US/Pacific](1), object(2)
memory usage: 29.6+ MB


# 3) Data Preprocessing

### a)Transposing ride timestamp dataset###

>Ride events are spread across rows for each ride. The records will be transposed for events to be across columns for easy analysis

In [ ]:
df_ride_timestamps.head()

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 02:39:19-07:00
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 02:39:51-07:00
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 02:44:31-07:00
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 02:44:33-07:00
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 03:03:05-07:00


In [ ]:
df_ride_timestamps = df_ride_timestamps.pivot(index='ride_id',columns='event',values='timestamp')
df_ride_timestamps.reset_index(inplace=True)
df_ride_timestamps.head()

event,ride_id,accepted_at,arrived_at,dropped_off_at,picked_up_at,requested_at
0,00003037a262d9ee40e61b5c0718f7f0,2016-06-13 09:39:51,2016-06-13 09:44:31,2016-06-13 10:03:05,2016-06-13 09:44:33,2016-06-13 09:39:19
1,00005eae40882760d675da5effb89ae3,2016-05-14 05:23:25,2016-05-14 05:26:17,2016-05-14 05:34:17,2016-05-14 05:26:18,2016-05-14 05:23:21
2,000061d42cf29f73b591041d9a1b2973,2016-05-16 15:43:14,2016-05-16 15:47:29,2016-05-16 15:54:18,2016-05-16 15:47:32,2016-05-16 15:43:09
3,00006efeb0d5e3ccad7d921ddeee9900,2016-05-11 19:29:43,2016-05-11 19:35:11,2016-05-11 19:40:47,2016-05-11 19:35:15,2016-05-11 19:29:36
4,0000d9b24d8ccdd991b76258e616fa01,2016-04-26 18:12:15,2016-04-26 18:16:52,2016-04-26 18:24:14,2016-04-26 18:16:55,2016-04-26 18:11:38


### b) Merging all three datasets###

**1. Merging Driver_Ids + ride_ids on driver id**

In [ ]:
df_driver_ids['driver_id'] = df_driver_ids['driver_id'].str.lower()
df_ride_ids['driver_id'] = df_ride_ids['driver_id'].str.lower()
df_driver_ride_merge = pd.merge(df_driver_ids,df_ride_ids,on='driver_id',how='outer')
df_driver_ride_merge.shape

(193584, 6)

In [ ]:
df_driver_ride_merge.loc[df_driver_ride_merge.ride_id.isnull()].shape

(83, 6)

>83 drivers were on-boarded, but they have no ride information. Either ride information is missing for them or they never completed any ride. The population of 83 out of 927 contributes to only 9% of data in driver_ids dataset and can be deleted.

--> Deleting 83 records where from driver_ids from

In [ ]:
df_driver_ride_merge.loc[df_driver_ride_merge.driver_onboard_date.isnull()].nunique()

driver_id                83
driver_onboard_date       0
ride_id                7611
ride_distance          5637
ride_duration          1922
ride_prime_time          10
dtype: int64

>83 drivers from ride_ids dataset are not present in onboarding dataset. Either their onboarding information is missing or they were onboarded before 28th March 2016. Considering exact 83 drivers from on-boarding dataset did not had ride information, it can be possible that driver id's between driver_ids and ride_ids dataset might have been mistakenly changed.

In [ ]:
df_driver_ride_merge.shape

(193584, 6)

In [ ]:
df_driver_ride_merge = pd.merge(df_driver_ids,df_ride_ids,on='driver_id',how='inner')
df_driver_ride_merge.shape

(185890, 6)

**2. Merging Driver_Ids + ride_ids + ride_timestamps on rideid**

In [ ]:
df_driver_ride_merge['ride_id'] = df_driver_ride_merge['ride_id'].str.lower()
df_ride_timestamps['ride_id'] = df_ride_timestamps['ride_id'].str.lower()
df_final_merge = pd.merge(df_driver_ride_merge,df_ride_timestamps,on='ride_id',how='outer')
df_final_merge.shape

(195763, 11)

In [ ]:
df_final_merge.loc[df_final_merge.accepted_at.isnull()].nunique()

driver_id                18
driver_onboard_date      14
ride_id                1683
ride_distance          1555
ride_duration           990
ride_prime_time           9
accepted_at               0
arrived_at                0
dropped_off_at            0
picked_up_at              0
requested_at              0
dtype: int64

>1683 rides for 18 drivers are not captured in ride_timestamp dataset. As we cannot relate this population to ride information these records will be deleted.

In [ ]:
df_final_merge.loc[df_final_merge.driver_id.isnull()].nunique()

driver_id                 0
driver_onboard_date       0
ride_id                9873
ride_distance             0
ride_duration             0
ride_prime_time           0
accepted_at            9863
arrived_at             9865
dropped_off_at         9856
picked_up_at           9864
requested_at           9863
dtype: int64

>9873 rides in ride_timestamp dataset does not have any information around drivers and rides and will be deleted

In [ ]:
df_final_merge = pd.merge(df_driver_ride_merge,df_ride_timestamps,on='ride_id',how='inner')
df_final_merge.shape

(184207, 11)

In [ ]:
pd.DataFrame(df_final_merge.nunique())

,0
driver_id,837
driver_onboard_date,49
ride_id,184207
ride_distance,27707
ride_duration,4063
ride_prime_time,12
accepted_at,180930
arrived_at,181010
dropped_off_at,180980
picked_up_at,180994


In [ ]:
df_final_merge.head()

,driver_id,driver_onboard_date,ride_id,ride_distance,ride_duration,ride_prime_time,accepted_at,arrived_at,dropped_off_at,picked_up_at,requested_at
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-04-23 02:14:15,2016-04-23 02:16:36,2016-04-23 02:22:07,2016-04-23 02:16:40,2016-04-23 02:13:50
1,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 19:00:52,2016-03-29 19:03:57,2016-03-29 19:17:30,2016-03-29 19:04:01,2016-03-29 19:00:49
2,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-06-21 11:56:39,2016-06-21 12:01:32,2016-06-21 12:11:07,2016-06-21 12:01:35,2016-06-21 11:56:31
3,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-05-19 09:15:33,2016-05-19 09:18:20,2016-05-19 10:13:58,2016-05-19 09:18:20,2016-05-19 09:15:29
4,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-04-20 22:05:32,2016-04-20 22:07:03,2016-04-20 22:20:45,2016-04-20 22:07:02,2016-04-20 22:05:30


### c) Calculating fares for each ride###

In [ ]:
df_fare_calculations = df_final_merge

In [ ]:
# Converting ride distance from meters to miles
df_fare_calculations['ride_distance_miles'] = df_fare_calculations.apply(lambda row: row.ride_distance * 0.000621371, axis =1)
# Converting ride duration from seconds to minutes
df_fare_calculations['ride_duration_minutes'] = df_fare_calculations.apply(lambda row: row.ride_duration / 60, axis =1)

In [ ]:
# Converting drivers earnings
df_fare_calculations['driver_earning'] = df_fare_calculations.apply(lambda row: ( ((2 + row.ride_distance_miles * 1.15 + row.ride_duration_minutes * 0.22) * (1 + row.ride_prime_time/100)))*0.80, axis =1)
df_fare_calculations['driver_earning'] = df_fare_calculations['driver_earning'].apply(lambda x: 5 if x < 5 else x)
df_fare_calculations['driver_earning'] = df_fare_calculations['driver_earning'].apply(lambda x: 400 if x > 400 else x)

In [ ]:
# Converting drivers earnings
df_fare_calculations['lyft_earning'] = df_fare_calculations.apply(lambda row: 1.75 + ( (2 + row.ride_distance_miles * 1.15 + row.ride_duration_minutes * 0.22) * (1 + (row.ride_prime_time /100)) * 0.20  ), axis =1)

In [ ]:
# Convering event timestamp to timestamp format
df_fare_calculations['accepted_at'] = pd.to_datetime(df_fare_calculations['accepted_at'])
df_fare_calculations['dropped_off_at'] = pd.to_datetime(df_fare_calculations['dropped_off_at'])
df_fare_calculations['picked_up_at'] = pd.to_datetime(df_fare_calculations['picked_up_at'])
df_fare_calculations['requested_at'] = pd.to_datetime(df_fare_calculations['requested_at'])

In [ ]:
# Coverting dropped off timestamp to week number
df_fare_calculations['dropped_off_at_weekno'] = df_fare_calculations['dropped_off_at'].dt.week

In [ ]:
# Calculating time difference in secs between ride requested at and accepted at
df_fare_calculations['requested_accepted_gap_sec'] = (df_fare_calculations['accepted_at'] - df_fare_calculations['requested_at']).dt.seconds

# Calculating time difference in secs between ride accepted at and ride arrived at
df_fare_calculations['accepted_arrived_gap_sec'] = (pd.to_datetime(df_fare_calculations['arrived_at']) - df_fare_calculations['accepted_at']).dt.seconds

In [ ]:
df_fare_calculations.head()

,driver_id,driver_onboard_date,ride_id,ride_distance,ride_duration,ride_prime_time,accepted_at,arrived_at,dropped_off_at,picked_up_at,requested_at,ride_distance_miles,ride_duration_minutes,driver_earning,lyft_earning,dropped_off_at_weekno,requested_accepted_gap_sec,accepted_arrived_gap_sec
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,2016-04-23 02:14:15,2016-04-23 02:16:36,2016-04-23 02:22:07,2016-04-23 02:16:40,2016-04-23 02:13:50,1.125303,5.450000,5.391718,3.097929,16,25,141
1,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,2016-03-29 19:00:52,2016-03-29 19:03:57,2016-03-29 19:17:30,2016-03-29 19:04:01,2016-03-29 19:00:49,2.089049,13.483333,5.894992,3.223748,13,3,185
2,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,029227c4c2971ce69ff2274dc798ef43,3282,572,0,2016-06-21 11:56:39,2016-06-21 12:01:32,2016-06-21 12:11:07,2016-06-21 12:01:35,2016-06-21 11:56:31,2.039340,9.533333,5.154059,3.038515,25,8,293
3,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,034e861343a63ac3c18a9ceb1ce0ac69,65283,3338,25,2016-05-19 09:15:33,2016-05-19 09:18:20,2016-05-19 10:13:58,2016-05-19 09:18:20,2016-05-19 09:15:29,40.564963,55.633333,60.889041,16.972260,20,4,167
4,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00,034f2e614a2f9fc7f1c2f77647d1b981,4115,823,100,2016-04-20 22:05:32,2016-04-20 22:07:03,2016-04-20 22:20:45,2016-04-20 22:07:02,2016-04-20 22:05:30,2.556942,13.716667,12.733039,4.933260,16,2,91


## d) Summarizing data##

**1. Summary by drivers per week**

In [ ]:
df_summ_driver_weekly = df_fare_calculations[['driver_id','ride_id','ride_distance_miles','ride_duration_minutes','ride_prime_time','driver_earning','lyft_earning','dropped_off_at_weekno','dropped_off_at','driver_onboard_date','requested_accepted_gap_sec','accepted_arrived_gap_sec']].groupby(['driver_id','dropped_off_at_weekno']).agg({'ride_id':['count'],'ride_distance_miles':['mean'],'ride_duration_minutes':['mean'],'ride_prime_time':['mean'],'driver_earning':['sum'],'lyft_earning':['sum'],'dropped_off_at':['max'],'driver_onboard_date':['max'],'requested_accepted_gap_sec':['mean'],'accepted_arrived_gap_sec':['mean']})
df_summ_driver_weekly.columns = ["_".join(x) for x in df_summ_driver_weekly.columns.ravel()]
df_summ_driver_weekly.reset_index(inplace=True)
df_summ_driver_weekly.head()

,driver_id,dropped_off_at_weekno,ride_id_count,ride_distance_miles_mean,ride_duration_minutes_mean,ride_prime_time_mean,driver_earning_sum,lyft_earning_sum,dropped_off_at_max,driver_onboard_date_max,requested_accepted_gap_sec_mean,accepted_arrived_gap_sec_mean
0,002be0ffdc997bd5c50703158b7c2491,13,26,2.539902,10.542308,8.653846,171.444885,86.308688,2016-04-02 02:21:12,2016-03-29 00:00:00,7.961538,163.307692
1,002be0ffdc997bd5c50703158b7c2491,14,11,3.567799,10.818182,29.545455,100.996874,43.506943,2016-04-10 02:17:55,2016-03-29 00:00:00,46.909091,161.181818
2,002be0ffdc997bd5c50703158b7c2491,15,43,3.904580,12.600388,17.441860,369.726658,165.337680,2016-04-17 02:03:07,2016-03-29 00:00:00,4.627907,158.930233
3,002be0ffdc997bd5c50703158b7c2491,16,29,3.900860,12.744828,23.275862,267.945460,116.214878,2016-04-24 02:19:26,2016-03-29 00:00:00,5.310345,164.517241
4,002be0ffdc997bd5c50703158b7c2491,17,3,1.480520,8.900000,8.333333,16.760030,8.878695,2016-04-27 17:24:45,2016-03-29 00:00:00,12.333333,102.000000


**2. Summary by drivers**

In [ ]:
df_summ_driver = df_summ_driver_weekly[['driver_id','ride_id_count','ride_distance_miles_mean','ride_duration_minutes_mean','ride_prime_time_mean','driver_earning_sum','lyft_earning_sum','dropped_off_at_max','driver_onboard_date_max','requested_accepted_gap_sec_mean','accepted_arrived_gap_sec_mean']].groupby(['driver_id']).agg({'ride_id_count':['sum'],'ride_distance_miles_mean':['mean'],'ride_duration_minutes_mean':['mean'],'ride_prime_time_mean':['mean'],'driver_earning_sum':['sum'],'lyft_earning_sum':['sum'],'dropped_off_at_max':['max'],'driver_onboard_date_max':['max'],'requested_accepted_gap_sec_mean':['mean'],'accepted_arrived_gap_sec_mean':['mean']})
df_summ_driver.columns = df_summ_driver.columns.droplevel(1)
df_summ_driver.reset_index(inplace=True)

In [ ]:
df_summ_driver.head()

,driver_id,ride_id_count,ride_distance_miles_mean,ride_duration_minutes_mean,ride_prime_time_mean,driver_earning_sum,lyft_earning_sum,dropped_off_at_max,driver_onboard_date_max,requested_accepted_gap_sec_mean,accepted_arrived_gap_sec_mean
0,002be0ffdc997bd5c50703158b7c2491,277,3.685312,12.768153,18.647499,2526.858128,1099.909163,2016-06-23 10:29:53,2016-03-29 00:00:00,10.798463,167.430474
1,007f0389f9c7b03ef97098422f902e62,31,2.313997,10.963333,20.648148,223.188647,107.698933,2016-06-22 13:28:38,2016-03-29 00:00:00,10.488889,205.518519
2,011e5c5dfc5c2c92501b8b24d47509bc,34,6.045286,15.736414,19.754464,345.327005,144.085558,2016-06-12 20:30:38,2016-04-05 00:00:00,7.409821,212.032143
3,0152a2f305e71d26cc964f8d4411add9,191,4.798184,15.159162,11.124694,1869.835875,789.179684,2016-06-26 10:36:13,2016-04-23 00:00:00,14.829329,235.496884
4,01674381af7edd264113d4e6ed55ecda,375,5.377523,16.144313,11.474489,3854.045562,1601.193268,2016-06-24 13:27:38,2016-04-29 00:00:00,13.128968,215.301245


In [ ]:
df_summ_driver.describe()

,ride_id_count,ride_distance_miles_mean,ride_duration_minutes_mean,ride_prime_time_mean,driver_earning_sum,lyft_earning_sum,requested_accepted_gap_sec_mean,accepted_arrived_gap_sec_mean
count,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000,837.000000
mean,220.080048,4.606478,14.480658,15.748653,2129.618653,903.948134,11.895043,271.373761
std,178.180798,1.484297,2.099088,7.233765,1729.466974,730.537079,4.244357,89.336819
min,3.000000,2.049489,8.677341,0.000000,18.753044,9.119614,3.861280,96.133333
25%,44.000000,3.642367,13.128266,11.244048,441.540605,185.357656,9.408398,209.142207
50%,224.000000,4.255811,14.327222,15.578808,2110.177581,903.816520,10.999895,251.520546
75%,330.000000,5.225177,15.491650,19.607582,3215.911567,1362.812412,13.277955,311.586703
max,919.000000,13.365086,27.344382,58.119381,8964.367766,3756.365749,37.866837,751.022222


# 4) Modeling

## a) Calculating Drivers Lifetime Value##

>Drivers Lifetime Value is calculated as below: <br>
>>Step 1: Calculate churn rate <br>
>>Step 2: Calculate average lifespan of a driver <br>
>>Step 3: Calculate Lyft's marginal earning per driver per week <br>
>>Step 4: Multiple average lifespan of a driver to Lyft's marginal earning per driver per week

**Step 1: Calculating Churn Rate**

In [ ]:
df_weekly_summary = df_summ_driver_weekly

In [ ]:
def churn_model(churn_drop_parameter,churn_time_parameter,):

    # Determining running average
    df_weekly_summary['running_avg'] = ''
    for i in range(0,len(df_weekly_summary)):
        if i == 0:
            df_weekly_summary.loc[i,'running_avg'] = df_weekly_summary.loc[i,'driver_earning_sum']
        elif i == 1:
            df_weekly_summary.loc[i,'running_avg'] = df_weekly_summary.loc[i-1,'driver_earning_sum']
        elif i == 2:
            df_weekly_summary.loc[i,'running_avg'] = (df_weekly_summary.loc[i-2,'driver_earning_sum'] + df_weekly_summary.loc[i-1,'driver_earning_sum'])/2
        else:
            df_weekly_summary.loc[i,'running_avg'] = (df_weekly_summary.loc[i-3,'driver_earning_sum'] + df_weekly_summary.loc[i-2,'driver_earning_sum'] + df_weekly_summary.loc[i-1,'driver_earning_sum']) / 3

    # Determining drivers earning performance
    df_weekly_summary['driver_earning_performance'] = df_weekly_summary.apply(lambda row: row.driver_earning_sum - row.running_avg,axis=1)
    df_weekly_summary['driver_earning_performance_percent'] = df_weekly_summary.apply(lambda row: (row.driver_earning_performance/row.running_avg) * 100,axis=1)

    # Drivers churn indicatore is set if performance percent is less that churn drop parameter
    df_weekly_summary['driver_churn_indicator'] = df_weekly_summary['driver_earning_performance_percent'].apply(lambda x: 1 if x < churn_drop_parameter else 0)

    df_driver_summary = df_weekly_summary[['driver_id','ride_id_count','ride_distance_miles_mean','ride_duration_minutes_mean','ride_prime_time_mean','driver_earning_sum','lyft_earning_sum','dropped_off_at_max','driver_onboard_date_max','requested_accepted_gap_sec_mean','accepted_arrived_gap_sec_mean','driver_churn_indicator']].groupby(['driver_id']).agg({'ride_id_count':['sum'],'ride_distance_miles_mean':['mean'],'ride_duration_minutes_mean':['mean'],'ride_prime_time_mean':['mean'],'driver_earning_sum':['sum'],'lyft_earning_sum':['sum'],'dropped_off_at_max':['max'],'driver_onboard_date_max':['max'],'requested_accepted_gap_sec_mean':['mean'],'accepted_arrived_gap_sec_mean':['mean'],'driver_churn_indicator':['sum']})
    df_driver_summary.columns = df_driver_summary.columns.droplevel(1)
    df_driver_summary.reset_index(inplace=True)

    # Drivers churn flag is set if number of churns is more that churn time parameter
    df_driver_summary['driver_churn_flag'] = df_driver_summary['driver_churn_indicator'].apply(lambda x: True if x > churn_time_parameter else False)

    churn_rate =  (len(df_driver_summary[df_driver_summary.driver_churn_flag == True]) / df_driver_summary.shape[0])* 100

    return churn_rate, df_driver_summary

In [ ]:
churn_rate, df_driver_summary = churn_model(-40,3)
print('Average churn rate over 13 week:', churn_rate , 'percent')
print('Average churn rate per week:', churn_rate / 13, 'percent')

Average churn rate over 13 week: 20.78853046594982 percent
Average churn rate per week: 1.599117728149986 percent


**Step 2: Determining Average Lifespan of a driver**

In [ ]:
avg_driver_lifespan = (13 / churn_rate) * 100
print('Average life span of a driver (in weeks):', avg_driver_lifespan)

Average life span of a driver (in weeks): 62.53448275862069


**Step 3: Calculate Lyft's marginal earning per driver per week**

In [ ]:
lyft_total_earning = df_summ_driver['lyft_earning_sum'].sum()
number_of_weeks = abs((df_final_merge['dropped_off_at'].min() - df_final_merge['dropped_off_at'].max()).days / 7)
number_of_drivers = df_final_merge['driver_id'].nunique()
driver_avg_margin = lyft_total_earning / (number_of_weeks  * number_of_drivers)
print('Average margin earning from a driver per week:', driver_avg_margin)

Average margin earning from a driver per week: 69.53447185751139


**Step 4: Multiple average lifespan of a driver to Lyft's marginal earning per driver per week**

In [ ]:
driver_lifetime_value = avg_driver_lifespan * driver_avg_margin
print('Lifetime Value of a driver based on 13 weeks of data:',driver_lifetime_value)

Lifetime Value of a driver based on 13 weeks of data: 4348.302231503341


## b) Predictive Indicators for drivers churn##

In [ ]:
df_driver_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Data columns (total 13 columns):
driver_id                          837 non-null object
ride_id_count                      837 non-null int64
ride_distance_miles_mean           837 non-null float64
ride_duration_minutes_mean         837 non-null float64
ride_prime_time_mean               837 non-null float64
driver_earning_sum                 837 non-null float64
lyft_earning_sum                   837 non-null float64
dropped_off_at_max                 837 non-null datetime64[ns]
driver_onboard_date_max            837 non-null object
requested_accepted_gap_sec_mean    837 non-null float64
accepted_arrived_gap_sec_mean      837 non-null float64
driver_churn_indicator             837 non-null int64
driver_churn_flag                  837 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2), object(2)
memory usage: 79.4+ KB


In [ ]:
X = df_driver_summary.drop(['driver_id','driver_churn_flag','dropped_off_at_max','driver_onboard_date_max','driver_churn_indicator'], axis=1)
y = df_driver_summary['driver_churn_flag']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[95 33]
 [32  8]]
             precision    recall  f1-score   support

      False       0.75      0.74      0.75       128
       True       0.20      0.20      0.20        40

avg / total       0.62      0.61      0.61       168



In [ ]:
for name, importance in zip(X.columns, classifier.feature_importances_):
    print(name, importance)

ride_id_count 0.05376885740108105
ride_distance_miles_mean 0.10658322605866279
ride_duration_minutes_mean 0.10969675138010536
ride_prime_time_mean 0.22124064938473056
driver_earning_sum 0.09961910116971853
lyft_earning_sum 0.17076673747843593
requested_accepted_gap_sec_mean 0.06747665316365932
accepted_arrived_gap_sec_mean 0.17084802396360654


In [ ]:
df_driver_summary.head()

,driver_id,ride_id_count,ride_distance_miles_mean,ride_duration_minutes_mean,ride_prime_time_mean,driver_earning_sum,lyft_earning_sum,dropped_off_at_max,driver_onboard_date_max,requested_accepted_gap_sec_mean,accepted_arrived_gap_sec_mean,driver_churn_indicator,driver_churn_flag
0,002be0ffdc997bd5c50703158b7c2491,277,3.685312,12.768153,18.647499,2526.858128,1099.909163,2016-06-23 10:29:53,2016-03-29 00:00:00,10.798463,167.430474,5,True
1,007f0389f9c7b03ef97098422f902e62,31,2.313997,10.963333,20.648148,223.188647,107.698933,2016-06-22 13:28:38,2016-03-29 00:00:00,10.488889,205.518519,5,True
2,011e5c5dfc5c2c92501b8b24d47509bc,34,6.045286,15.736414,19.754464,345.327005,144.085558,2016-06-12 20:30:38,2016-04-05 00:00:00,7.409821,212.032143,1,False
3,0152a2f305e71d26cc964f8d4411add9,191,4.798184,15.159162,11.124694,1869.835875,789.179684,2016-06-26 10:36:13,2016-04-23 00:00:00,14.829329,235.496884,0,False
4,01674381af7edd264113d4e6ed55ecda,375,5.377523,16.144313,11.474489,3854.045562,1601.193268,2016-06-24 13:27:38,2016-04-29 00:00:00,13.128968,215.301245,1,False


In [ ]:
training_data2 = df_driver_summary[['driver_churn_flag','ride_id_count','ride_distance_miles_mean','ride_duration_minutes_mean','ride_prime_time_mean','driver_earning_sum','lyft_earning_sum','requested_accepted_gap_sec_mean','accepted_arrived_gap_sec_mean']].groupby(['driver_churn_flag']).agg({'ride_id_count':['count'],'ride_id_count':['mean'],'ride_distance_miles_mean':['mean'],'ride_duration_minutes_mean':['mean'],'ride_prime_time_mean':['mean'],'driver_earning_sum':['mean'],'lyft_earning_sum':['mean'],'requested_accepted_gap_sec_mean':['mean'],'accepted_arrived_gap_sec_mean':['mean']})
training_data2

,ride_id_count,ride_distance_miles_mean,ride_duration_minutes_mean,ride_prime_time_mean,driver_earning_sum,lyft_earning_sum,requested_accepted_gap_sec_mean,accepted_arrived_gap_sec_mean
,mean,mean,mean,mean,mean,mean,mean,mean
driver_churn_flag,,,,,,,,
False,235.592760,4.551973,14.441505,15.754629,2279.081470,967.484107,11.962354,265.813159
True,160.971264,4.814160,14.629844,15.725886,1560.113779,661.854169,11.638565,292.561569


## c) Driver Segmentation##

In [ ]:
training_data = df_summ_driver
training_data = training_data.drop(['driver_id','dropped_off_at_max','driver_onboard_date_max'],axis=1)

In [ ]:
# Getting statistics for a average driver
training_data.mean()

ride_id_count                       220.080048
ride_distance_miles_mean              4.606478
ride_duration_minutes_mean           14.480658
ride_prime_time_mean                 15.748653
driver_earning_sum                 2129.618653
lyft_earning_sum                    903.948134
requested_accepted_gap_sec_mean      11.895043
accepted_arrived_gap_sec_mean       271.373761
dtype: float64

In [ ]:
X = np.array(training_data.astype(float))

In [ ]:
kmeans = KMeans(n_clusters = 3)
kmeans = kmeans.fit(X)
training_data['label'] = kmeans.predict(X)

In [ ]:
# Grouping by each cluster group
training_data1 = training_data[['label','ride_id_count','ride_distance_miles_mean','ride_duration_minutes_mean','ride_prime_time_mean','driver_earning_sum','lyft_earning_sum','requested_accepted_gap_sec_mean','accepted_arrived_gap_sec_mean']].groupby(['label']).agg({'ride_id_count':['count'],'ride_id_count':['mean'],'ride_distance_miles_mean':['mean'],'ride_duration_minutes_mean':['mean'],'ride_prime_time_mean':['mean'],'driver_earning_sum':['mean'],'lyft_earning_sum':['mean'],'requested_accepted_gap_sec_mean':['mean'],'accepted_arrived_gap_sec_mean':['mean']})
training_data1.columns = training_data1.columns.droplevel(1)
training_data1.reset_index(inplace=True)

In [ ]:
# Getting statistics for a segmented driver
training_data1

,label,ride_id_count,ride_distance_miles_mean,ride_duration_minutes_mean,ride_prime_time_mean,driver_earning_sum,lyft_earning_sum,requested_accepted_gap_sec_mean,accepted_arrived_gap_sec_mean
0,0,510.268116,4.550998,14.729228,17.613337,4997.848527,2111.215963,11.689711,243.931139
1,1,43.804665,4.906716,14.514543,13.787591,438.271440,183.469735,12.124810,302.741180
2,2,277.429775,4.338709,14.351655,16.915277,2647.361269,1130.131085,11.753260,251.789650


In [ ]:
training_data.groupby('label').count()

,ride_id_count,ride_distance_miles_mean,ride_duration_minutes_mean,ride_prime_time_mean,driver_earning_sum,lyft_earning_sum,requested_accepted_gap_sec_mean,accepted_arrived_gap_sec_mean
label,,,,,,,,
0,138,138,138,138,138,138,138,138
1,343,343,343,343,343,343,343,343
2,356,356,356,356,356,356,356,356


## The End##